# Group 5 - Module 7: Dialogue systems and question answering

***
### Group Members:
* **Nils Dunlop, 20010127-2359, Applied Data Science, e-mail: gusdunlni@student.gu.se (15 hours)**
* **Francisco Erazo, 19930613-9214, Applied Data Science, e-mail: guserafr@student.gu.se (15 hours)**

#### **We hereby declare that we have both actively participated in solving every exercise. All solutions are entirely our own work, without having taken part of other solutions." (This is independent and additional to any declaration that you may encounter in the electronic submission system.)**

# Assignment 7
***

## Problem 1: Reading and Summary
***

The paper describes the development and functionality of GUS (Genial Understander System), a dialogue-based system designed primarily to simulate a travel agent's role for booking return trips within California. The system's main objective is to process and understand English dialogues, showcasing its capacity to manage various elements of natural conversation. These include handling shifts in initiative, interpreting indirect responses, resolving anaphora or expressions, and understanding conversational implicates.

GUS's architecture is modular, integrating multiple components such as morphological and syntactic analyzers and a frame reasoner. This structure allows it to execute dialogues efficiently through procedural attachment and scheduling techniques. The paper highlights the system's innovative approach to dialogue management, demonstrating its potential for understanding and generating human-like conversations within its specified domain.

Despite its advancements, the paper also discusses GUS's limitations and emphasizes the complexity of achieving realistic dialogue interactions, especially at the time of its publication in 1976. It points out the necessity for more sophisticated knowledge representation and reasoning capabilities to improve the system's performance. The development of GUS marks a significant step in the field of language understanding systems, illustrating both the potential and challenges of creating machines capable of engaging in meaningful dialogues with humans. Through its frame-driven design, GUS contributes to the ongoing research in artificial intelligence, particularly in natural language processing and understanding, setting a foundation for future improvements and innovations in the area.

## Problem 2: Implement A Simple Dialogue System
***

## References
***

- Choudhary, A. (2018). Reinforcement Learning Guide: Solving the Multi-Armed Bandit Problem from Scratch in Python. [online] Analytics Vidhya. Available at: https://www.analyticsvidhya.com/blog/2018/09/reinforcement-multi-armed-bandit-scratch-python/ [Accessed 23 Feb. 2024].

- Nested Software (2019). Tic-Tac-Toe with MCTS. [online] DEV Community. Available at: https://dev.to/nestedsoftware/tic-tac-toe-with-mcts-2h5k [Accessed 26 Feb. 2024].

- GfG (2019). ML Monte Carlo Tree Search (MCTS). [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/ml-monte-carlo-tree-search-mcts/ [Accessed 26 Feb. 2024].

## Self Check
***
- Have you answered all questions to the best of your ability?
Yes, we have.
- Is all the required information on the front page, is the file name correct etc.?
Indeed, all the required information on the front page has been included.
- Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?)
We have checked, and everything looks good.